<a href="https://colab.research.google.com/github/zenchu741/lottery/blob/master/lottery_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import openpyxl,random,pprint
import pandas as pd
import tkinter as tk
from tkinter import messagebox
import time, json, os
from tkinter import *
import threading
from configparser import RawConfigParser
from tkinter import ttk

In [15]:
class WinLottery:
  def __init__(self):
    id_file = ""
    reward_file = ""
    sheet_name = ""
    reward = ""
    num = 0
    going = bool
    is_run = bool

    deps = []
    it_rewards = {}
    dep_rewards = {}

    self.data_list = []
    self.frames = []

  def set_values(self):
    self.id_df = pd.read_excel(self.id_file, converters={'行編':str}, sheet_name=self.sheet_name)
    self.id_df['combine_all'] = self.id_df['行編'] + self.id_df['姓名'] + self.id_df['分機']
    # self.id_df['combine_all'] = self.id_df['combine_all'].str.replace(r'(80)', r' \1')
    self.id_df['combine_all'] = self.id_df['combine_all'].str[:-10] + ' ' + self.id_df['combine_all'].str[-10:]
    self.id_df['combine_no_phone'] = self.id_df['行編'] + self.id_df['姓名']
    self.data_list = self.id_df['combine_no_phone'].values.tolist()

  def check_already_reward(self):
    self.set_values()

    dep_list = self.deps

    reward_all = pd.Series()

    for dep in dep_list :
      ##讀取各部門獲獎名單，合併為一個欄位，加入至reward_all中
      reward_df = pd.read_excel(self.reward_file, sheet_name=dep)
      reward_df_combine = reward_df.stack().reset_index()
      reward_df_combine = reward_df_combine[0]
      reward_all = reward_all.append(reward_df_combine, ignore_index=True)

    return reward_all

  def get_data_list(self):

    self.set_values()

    IT_df_combine = self.id_df['combine_all'] ##資訊板塊人員名單


    reward_all = self.check_already_reward()

    ##檢查抽獎同仁名單中是否有獲獎名單的同仁，有則drop掉後回傳
    cond = IT_df_combine.isin(reward_all)
    IT_df_combine.drop(IT_df_combine[cond].index, inplace = True)
    data_list = IT_df_combine.reset_index(drop = True).tolist()

    return data_list		

  def write_to_excel(self, people_list):
    reward_df = pd.read_excel(self.reward_file, sheet_name=self.sheet_name)
    book = openpyxl.load_workbook(self.reward_file)
    sheet = book.get_sheet_by_name(self.sheet_name)
    writer = pd.ExcelWriter(self.reward_file, engine = 'openpyxl')
    writer.book = book

    if self.sheet_name == '資訊列車':
      reward_dict = self.it_rewards

    else:
      reward_dict = self.dep_rewards

    try:
      col = reward_dict[self.reward]
    except Exception as e:
      msg = '請確認抽獎獎項是否正確!'
      self.pupupmsg(msg)
      raise e
      

    check_null = reward_df[self.reward].isnull()
    exist_index = reward_df[self.reward][check_null].index
    exist_list = reward_df[self.reward].tolist()

    if len(exist_list) != 0 and len(exist_index) == 0:
      index = len(exist_list)

    elif len(exist_list) != 0 :
      index = exist_index[0]

    else:
      index = 0

    for i in range(0,len(people_list)):
      cellref = sheet.cell(row=index+2, column=col)
      cellref.value = people_list[i]
      index += 1

    writer.save()				


  def lottery_roll(self, var1, var2,  window):
    self.set_values()

    filenames = ["train1.txt", "train2.txt"]

    for name in filenames:
      with open(name, 'r') as f:
        # print(f.read())
        # frames.append(f.readlines())
        self.frames.append(f.read())
    show_member = random.choice(self.data_list)
    train =  random.choice(self.frames)
    var1.set(train)
    var2.set(show_member)

    if self.going:
      window.after(30, self.lottery_roll, var1, var2, window)

    else:
      var2.set('恭喜以下中獎同仁！')


  def get_lottery(self, num):
    try:
      data_list = self.get_data_list()

    except Exception as e:
      msg = '請確認部門名稱是否正確!'
      self.pupupmsg(msg)
      raise e

    win_people_list_phone = random.sample(data_list, num)
    win_people_list = []

    for w in win_people_list_phone:
      
      win_people_info = w.split(' ')
      # print(win_people_info)
      win_people_list.append(win_people_info[0])

    self.write_to_excel(win_people_list_phone)


    return win_people_list

  def pupupmsg(self, msg):
    messagebox.showinfo('提示', msg)

In [14]:
w = WinLottery()
w.set_values()

AttributeError: ignored